In [1]:
import h5rdmtoolbox as h5tbx

Failed to import module h5tbx


In [2]:
from ontolutils import SSNO, Thing

In [3]:
from pivmetalib import ssno

In [4]:
snt = ssno.StandardNameTable(
    hasStandardNames=[ssno.StandardName(description='a desc'),
                      ssno.StandardName(description='b desc')])
snt

StandardNameTable(hasStandardNames=[StandardName(description=a desc), StandardName(description=b desc)])

In [5]:
yvel = ssno.StandardName(standard_name='y_velocity',
                         description='A y vel',
                         canonicalUnits='m/s',
                         standard_name_table=ssno.StandardNameTable(identifier='https://doi.org/10.5281/zenodo.10428817'))
yvel

StandardName(canonical_units=http://qudt.org/vocab/unit/M-PER-SEC, standard_name=y_velocity, description=A y vel, standard_name_table=StandardNameTable("None"))

In [6]:
xvel = ssno.StandardName(standard_name='x_velocity',
                         description='A x vel',
                         canonicalUnits='m/s',
                         standard_name_table=ssno.StandardNameTable(identifier='https://doi.org/10.5281/zenodo.10428817'))
xvel

StandardName(canonical_units=http://qudt.org/vocab/unit/M-PER-SEC, standard_name=x_velocity, description=A x vel, standard_name_table=StandardNameTable("None"))

In [7]:
xvel = ssno.StandardName(standard_name_table=ssno.StandardNameTable(identifier='https://doi.org/10.5281/zenodo.10428817'))
yvel = ssno.StandardName(standard_name_table=ssno.StandardNameTable(identifier='https://doi.org/10.5281/zenodo.10428817'))

In [8]:
# thing = Thing(canonical_units='m/s', description='The x velocity')
# thing.urirefs['canonical_units'] = 'ex:canonical_units'
# thing.namespaces['ex'] = 'https://example.org/'

In [9]:
with h5tbx.File(mode='w') as h5:
    ds = h5.create_dataset('u', data=3)
    ds.attrs['standard_name', SSNO.hasStandardName] = 'xvel'
    ds.rdf.object['standard_name'] = xvel
    # ds.rdf.object['standard_name'] = snt
    
    ds = h5.create_dataset('v', data=3)
    ds.attrs['standard_name', SSNO.hasStandardName] = 'y_velocity'
    ds.rdf.object['standard_name'] = yvel
    h5.dump(collapsed=False)

hdf_filename = h5.hdf_filename

C:\Users\da4323\Anaconda3\envs\h5tbx\lib\site-packages\pydantic\main.py:314: UserWarning: Pydantic serializer warnings:
  Expected `url` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [10]:
from pprint import pprint

In [11]:
with h5tbx.File(hdf_filename, mode='r') as h5:
    pprint(h5.u.rdf.object['standard_name'])

{'@context': {'dcat': 'http://www.w3.org/ns/dcat#',
              'dcterms': 'http://purl.org/dc/terms/',
              'owl': 'http://www.w3.org/2002/07/owl#',
              'prov': 'http://www.w3.org/ns/prov#',
              'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
              'skos': 'http://www.w3.org/2004/02/skos/core#',
              'ssno': 'https://matthiasprobst.github.io/ssno#',
              'standard_name_table': 'https://matthiasprobst.github.io/ssno#standardNameTable'},
 '@type': 'ssno:StandardName',
 'standard_name_table': {'@type': 'ssno:StandardNameTable'}}


In [12]:
from rdflib.plugins.shared.jsonld.context import Context

In [13]:
jsonld = h5tbx.dump_jsonld(h5.hdf_filename,
                           structural=True,
                           compact=True,
                           resolve_keys=True,
                           context={'ssno': 'https://matthiasprobst.github.io/ssno#'})

Find dataset with standard name "x_velocity"

In [14]:
import rdflib

def find_dataset_with_standard_name(standard_name: str):
    s = f"""PREFIX ssno: <https://matthiasprobst.github.io/ssno#>
    PREFIX hdf5: <http://purl.allotrope.org/ontologies/hdf5/1.8#>
    
    SELECT ?h5name
    WHERE {{
        ?h5obj a hdf5:Dataset .
        ?h5obj hdf5:name ?h5name .
        ?h5obj ssno:hasStandardName ?sn .
        ?sn a ssno:StandardName .
        ?sn ssno:standard_name "{standard_name}" .
    }}
    """
    g = rdflib.Graph().parse(data=jsonld, format='json-ld')
    return [r[0] for r in g.query(s)]

def find_one_dataset_with_standard_name(standard_name: str):
    s = f"""PREFIX ssno: <https://matthiasprobst.github.io/ssno#>
    PREFIX hdf5: <http://purl.allotrope.org/ontologies/hdf5/1.8#>
    
    SELECT ?h5name
    WHERE {{
        ?h5obj a hdf5:Dataset .
        ?h5obj hdf5:name ?h5name .
        ?h5obj ssno:hasStandardName ?sn .
        ?sn a ssno:StandardName .
        ?sn ssno:standard_name "{standard_name}" .
    }}
    """
    g = rdflib.Graph().parse(data=jsonld, format='json-ld')
    for r in g.query(s):
        return str(r[0])

def find_dataset_with_part_standard_name(substring_name: str):
    s = f"""PREFIX ssno: <https://matthiasprobst.github.io/ssno#>
    PREFIX hdf5: <http://purl.allotrope.org/ontologies/hdf5/1.8#>
    
    SELECT ?h5name
    WHERE {{
        ?h5obj a hdf5:Dataset .
        ?h5obj hdf5:name ?h5name .
        ?h5obj ssno:hasStandardName ?sn .
        ?sn a ssno:StandardName .
        ?sn ssno:standard_name ?standard_name .
        FILTER (CONTAINS(?standard_name, "{substring_name}"))
    }}
    """
    g = rdflib.Graph().parse(data=jsonld, format='json-ld')
    return [str(r[0]) for r in g.query(s)]

In [15]:
# print(jsonld)

In [16]:
class JSONLD:
    def __init__(self, filename):
        self.jsonld = h5tbx.dump_jsonld(filename)
        self.g = rdflib.Graph().parse(data=self.jsonld, format='json-ld')

    def find_one_dataset_with_standard_name(self, standard_name: str):
        """This finds the dataset based on the stanard name attribute string or its StandardName instance"""
        for r in self._find_dataset_with_standard_name(standard_name):
            return str(r[0])

    def find_dataset_with_standard_name(self, standard_name: str):
        """This finds the dataset based on the stanard name attribute string or its StandardName instance"""
        return [str(r[0]) for r in self._find_dataset_with_standard_name(standard_name)]

    def _find_dataset_with_standard_name(self, standard_name: str):
        """This finds the dataset based on the stanard name attribute string or its StandardName instance"""
        s = f"""PREFIX ssno: <https://matthiasprobst.github.io/ssno#>
        PREFIX hdf5: <http://purl.allotrope.org/ontologies/hdf5/1.8#>
        
        SELECT ?name
        WHERE {{
        ?subject a hdf5:Dataset ;
            hdf5:name ?name ;
          {{ ?subject ssno:hasStandardName "{standard_name}" . }}
          UNION
          {{ ?subject  ssno:hasStandardName ?sn .
             ?sn a ssno:StandardName ;
                 ssno:standard_name "{standard_name}" .
          }}
        }}
        """
        return self.g.query(s)
            
    def destinct_standard_names(self):
        s = """PREFIX ssno: <https://matthiasprobst.github.io/ssno#>
        PREFIX hdf5: <http://purl.allotrope.org/ontologies/hdf5/1.8#>
        
        SELECT ?standard_name
        WHERE {
            ?subject a hdf5:Dataset ;
                     ssno:hasStandardName ?sn .
            ?sn a ssno:StandardName ;
                     ssno:standard_name ?standard_name
        }
        """
        return [str(r[0]) for r in self.g.query(s)]

In [17]:
JSONLD(hdf_filename).find_dataset_with_standard_name('xvel')

['/u']

In [18]:
JSONLD(hdf_filename).find_one_dataset_with_standard_name('xvel')

'/u'

In [19]:
JSONLD(hdf_filename).find_one_dataset_with_standard_name('x_velocity')

In [20]:
JSONLD(hdf_filename).find_one_dataset_with_standard_name('y_velocity')

'/v'

In [21]:
JSONLD(hdf_filename).destinct_standard_names()

[]